## Save model to S3 storage

### Import Caikit-NLP

In [1]:
!git clone https://github.com/caikit/caikit-nlp.git

Cloning into 'caikit-nlp'...
remote: Enumerating objects: 5954, done.
remote: Counting objects: 100% (643/643), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 5954 (delta 444), reused 546 (delta 403), pack-reused 5311
Receiving objects: 100% (5954/5954), 2.56 MiB | 27.03 MiB/s, done.
Resolving deltas: 100% (4361/4361), done.


In [2]:
!pip install ./caikit-nlp

Processing ./caikit-nlp
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 221.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 331.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 320.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 305.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.9/419.9 kB 304.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 158.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 258.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 334.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 227.9

In [18]:
import os
import time
import boto3
import botocore

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import caikit_nlp

<function register_backend_type at 0x7f4aa20b7820> is still in the BETA phase and subject to change!


### Load model from HuggingFace Hub and save locally

In [19]:
output_dir = "../models/opt-350m_DETOXIFY_CAUSAL_LM"

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
peft_model = PeftModel.from_pretrained(model, "exyou/opt-350m_DETOXIFY_CAUSAL_LM")
peft_model = peft_model.merge_and_unload()

peft_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('../models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer_config.json',
 '../models/opt-350m_DETOXIFY_CAUSAL_LM/special_tokens_map.json',
 '../models/opt-350m_DETOXIFY_CAUSAL_LM/vocab.json',
 '../models/opt-350m_DETOXIFY_CAUSAL_LM/merges.txt',
 '../models/opt-350m_DETOXIFY_CAUSAL_LM/added_tokens.json',
 '../models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer.json')

### Convert model to Caikit format

In [21]:
input_dir = "../models/opt-350m_DETOXIFY_CAUSAL_LM"

output_dir = "../models/opt-350m_DETOXIFY_CAUSAL_LM-caikit"

start_time = time.time()
model = caikit_nlp.text_generation.TextGeneration.bootstrap(input_dir)
model.save(model_path=output_dir)
end_time = time.time()
print(f"Time to convert model: {end_time - start_time:.5f} seconds")

Time to convert model: 9.06516 seconds


### Upload Caikit model to S3 storage

In [22]:
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)

def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [25]:
upload_directory_to_s3("../models", "models")

../models/opt-350m_DETOXIFY_CAUSAL_LM/model.safetensors -> models/opt-350m_DETOXIFY_CAUSAL_LM/model.safetensors
../models/opt-350m_DETOXIFY_CAUSAL_LM/merges.txt -> models/opt-350m_DETOXIFY_CAUSAL_LM/merges.txt
../models/opt-350m_DETOXIFY_CAUSAL_LM/config.json -> models/opt-350m_DETOXIFY_CAUSAL_LM/config.json
../models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer.json -> models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer.json
../models/opt-350m_DETOXIFY_CAUSAL_LM/special_tokens_map.json -> models/opt-350m_DETOXIFY_CAUSAL_LM/special_tokens_map.json
../models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer_config.json -> models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer_config.json
../models/opt-350m_DETOXIFY_CAUSAL_LM/vocab.json -> models/opt-350m_DETOXIFY_CAUSAL_LM/vocab.json
../models/opt-350m_DETOXIFY_CAUSAL_LM/generation_config.json -> models/opt-350m_DETOXIFY_CAUSAL_LM/generation_config.json
../models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/config.yml -> models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/config.yml
../model

In [26]:
# sanity check
list_objects("models")

models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/config.json
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/generation_config.json
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/merges.txt
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/model.safetensors
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/special_tokens_map.json
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/tokenizer.json
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/tokenizer_config.json
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/artifacts/vocab.json
models/opt-350m_DETOXIFY_CAUSAL_LM-caikit/config.yml
models/opt-350m_DETOXIFY_CAUSAL_LM/config.json
models/opt-350m_DETOXIFY_CAUSAL_LM/generation_config.json
models/opt-350m_DETOXIFY_CAUSAL_LM/merges.txt
models/opt-350m_DETOXIFY_CAUSAL_LM/model.safetensors
models/opt-350m_DETOXIFY_CAUSAL_LM/special_tokens_map.json
models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer.json
models/opt-350m_DETOXIFY_CAUSAL_LM/tokenizer_config.json
models/opt-350m_DETOXIFY_